In [190]:
import io
import re
import csv
import pandas as pd
import win32com.client
from docx import Document

In [191]:
PathQ = "Q.docx"

In [192]:
prov = open('../provinsi.txt', 'r').read().lower()

In [193]:
prov = prov.split('\n')

In [194]:
prov = set(prov[:len(prov)-1])

In [195]:
docQ = Document(open(PathQ, 'rb'))

In [196]:
i = 0
par_1 = []
for idx, para in enumerate(docQ.paragraphs):
    if (para.text.lower() not in prov) and (len(para.text.strip(' \n\r\t')) != 0):
        tes = i, para
        par_1.append(tes)
        i = i+1

In [197]:
i = 0
y = 0
par = []
kab = []
kab_bold = []
for idx, para in par_1:
#     print(para)
    if idx in kab_bold:
        continue
    temp = {}
#     if i == 0:
#         kab[i]['end_pos'] = 0
    if idx == len(par_1)-1:
        kab[i-1]['end_pos'] = idx
    for r in para.runs:
        if r.font.size == 114300:
            temp_kabkot = para.text
            cek_tit = True
            xx = idx + 1
            if(xx == len(par_1)):
                cek_tit = False
            while cek_tit:
                temp_kabkot2 = [run.text for run in par_1[xx][1].runs if run.bold]
                if len(temp_kabkot2) > 0:
                    kab_bold.append(xx)
                    temp_kabkot = temp_kabkot +' '+temp_kabkot2[0]
                    xx = xx + 1
                elif(xx == len(par_1)):
                    cek_tit = False
                else:
                    cek_tit = False
            
#             print(temp_kabkot)
#             dist = re.search(r'KOTA |KAB. ',temp_kabkot).group()
#             nama = re.sub(r'KOTA |KAB. ','', temp_kabkot)
#             nama = re.sub(r'\s+', '',nama)
            temp['text'] = temp_kabkot
            if (i != 0) and (idx != len(par_1)-1):
                kab[i-1]['end_pos'] = idx-2
            kab.append(temp)
            i = i+1
        else:
            temp2 = y, para
            par.append(temp2)
            y = y+1

In [198]:
kab = pd.DataFrame(kab)

In [199]:
kab.head()

,end_pos,text
0,5,KAB. SIMEULUE
1,20,KAB. ACEH SINGKIL
2,25,KAB. ACEH TENGGARA
3,32,KAB. ACEH TIMUR
4,36,KAB. ACEH TENGAH


In [200]:
len(par)

18096

In [201]:
batas = 0
pars = []
title_bold = []
for idx, para in par:
    if idx in title_bold:
        continue
    paratemp = []
    for run in para.runs:
        if run.bold:
            temptit = [run.text for run in para.runs if run.bold][0]
            cek_tit = True
            xx = idx + 1
#             print(xx)
            if(xx == len(par)):
                cek_tit = False
            while cek_tit:
                temptit2 = [run.text for run in par[xx][1].runs if run.bold]
                if len(temptit2) > 0:
                    title_bold.append(xx)
                    temptit = temptit +' '+temptit2[0]
                    xx = xx + 1
                    if(xx == len(par)):
                        cek_tit = False
                else:
                    cek_tit = False
            paratemp.append(temptit+' [NAMA_PERUSAHAAN]')

            for x in range(batas, idx):
                for run in par[x][1].runs:
                    if len(run.text.strip(' \n\r\t')) != 0:
                        if run.bold:
                            continue
                        else:
                            if len(kab) > 0:
                                if (re.search(r'[^\-]\b[\d]{5}\b', par[x][1].text)) and (re.search(r'Ext: \b[\d]{5}\b', par[x][1].text) == None):
                                    kode_pos = re.search(r'[^\-]\b[\d]{5}\b', par[x][1].text).group().strip()
                                    alamat = re.sub(r'[^\-]\b[\d]{5}\b', '',  par[x][1].text)
                                    temp_alamat1 = alamat + ', '+ kab[kab.end_pos <= idx].tail(1).iloc[0].text + ' '+kode_pos
#                                 elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', par[x][1].text)):
#                                     temp_alamat1 = par[x][1].text + ', '+ kab[kab.end_pos <= idx].tail(1).iloc[0].text
                                else:
                                    temp_alamat1 = par[x][1].text
                            else:
                                temp_alamat1 = par[x][1].text
                            paratemp.append(temp_alamat1)
            
            pars.append(paratemp)
            batas = xx

IndexError: list index out of range

In [189]:
par[18095][1].text

'XVII CENDERAWASIH'

In [178]:
[run.text for run in par[18095][1].runs if run.bold]

['XVII CENDERAWASIH']

In [94]:
par_temp[:20]

[['RSUD RSUD KAB. ACEH SINGKIL [NAMA_PERUSAHAAN]',
  '0650-8001070',
  '0650-8001070',
  'rsud_kab.simeulue@yahoo.co.id',
  'DSN CEMARA INDAH, AMERIA BAHAGIA, KEC. KAB. SIMEULUE'],
 ['KLINIK DELIMA MAKMUR, PT [NAMA_PERUSAHAAN]', '081260252321'],
 ['KLINIK PERKEBUNAN [NAMA_PERUSAHAAN]',
  'MAKMUR, KEC. DANAU PARIS KAB. SIMEULUE 082304449035'],
 ['RSUD ACEH SINGKIL [NAMA_PERUSAHAAN]',
  '23784',
  '0658-21878',
  '0658-21878',
  'rsudacehsingkil@yahoo.com'],
 ['RSUD H HASANUDIN [NAMA_PERUSAHAAN]', '23784'],
 ['RSUD DR ZUBIR MAHMUD [NAMA_PERUSAHAAN]',
  'JL KUTACANE BELANG KEJEREN, TANAH MERAH, KAB. ACEH TENGGARA KEC. BADAR KAB. ACEH TENGGARA 24652 0646-21146 Ext: 2'],
 ['BLUD DATU BERU TAKENGON RSUD DR ZUBIR MAHMUD KAB. ACEH TENGAH [NAMA_PERUSAHAAN]',
  'rsud.idi@gmail.com',
  'JL MEDAN BANDA ACEH KM 386, SEUNEUBOK KAB. ACEH TIMUR'],
 ['RSUD CUT NYAK DHIEN [NAMA_PERUSAHAAN]',
  'JL RSUD DATU BERU, BUKIT, KEC. KEBAYAKAN KAB. ACEH TENGAH 24519'],
 ['RSUS MONTELLA [NAMA_PERUSAHAAN]',
  '065

In [92]:
# menggabungkan alamat
par_temp = []
for idx, p in enumerate(pars):
    parss = []
    batas = len(p)-1
    batas_bawah = []
    for idxx, x in enumerate(p):
        if idxx in batas_bawah:
            continue
        tempAlamat = ''
        if idxx <= batas:
            if re.search(r'( \[NAMA_PERUSAHAAN\])', x):
                parss.append(x)
            elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                if idxx == len(p)-1:
                    tempAlamat = x
                else:
                    xxx = idxx
                    while (xxx <= len(p)-1):
                        if (re.search(r'[^\-]\b[\d]{5}\b', p[xxx])):
                            batas_bawah.append(xxx)
                            tempAlamat = tempAlamat + ' '+ p[xxx]
                            xxx = xxx + 1
                        elif (re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', x)):
                            batas_bawah.append(xxx)
                            tempAlamat = tempAlamat + ' '+ p[xxx]
                            xxx = xxx + 1
                parss.append(tempAlamat.strip(' \n\r\t'))
            else:
                parss.append(x)
    par_temp.append(parss)

In [93]:
par_temp[:5]

[['RSUD RSUD KAB. ACEH SINGKIL [NAMA_PERUSAHAAN]',
  '0650-8001070',
  '0650-8001070',
  'rsud_kab.simeulue@yahoo.co.id',
  'DSN CEMARA INDAH, AMERIA BAHAGIA, KEC. KAB. SIMEULUE'],
 ['KLINIK DELIMA MAKMUR, PT [NAMA_PERUSAHAAN]', '081260252321'],
 ['KLINIK PERKEBUNAN [NAMA_PERUSAHAAN]',
  'MAKMUR, KEC. DANAU PARIS KAB. SIMEULUE 082304449035'],
 ['RSUD ACEH SINGKIL [NAMA_PERUSAHAAN]',
  '23784',
  '0658-21878',
  '0658-21878',
  'rsudacehsingkil@yahoo.com'],
 ['RSUD H HASANUDIN [NAMA_PERUSAHAAN]', '23784']]

In [75]:
# Buat DF
data = []
tempTel1 = ''
tempTel2 = ''
tempTel3 = ''  
tempEmail = ''
tempWeb = ''
for line in par_temp:

    row = {'Kategori': 'AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL',
           'Telp1': '', 
           'Telp2': '', 
           'Telp3': '',
           'Email': '',
           'Website': '',
           'Alamat': '',
           'Nama Perusahaan': '',
           'Kode Kategori': '',
           'Status Badan Usaha': ''}

    for text in line:
        if(re.search(r'[0-9]{3,}-\d{5,}|^08|\+62|\d{3}-62-\d{2,3}-\d{5,}|--\d{5,}|\d{6,}', text)):
            if(tempTel1 == ''):
                tempTel1 = text
                row['Telp1'] = tempTel1
            elif(tempTel2 == ''):
                tempTel2 = text
                row['Telp2'] = tempTel2
            else:
                tempTel3 = text
                row['Telp3'] = tempTel3
        elif(re.search(r'( \[NAMA_PERUSAHAAN\])', text)):
#             print(text)
            text = re.sub(r'( \[NAMA_PERUSAHAAN\])', '', text)
            if (re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text)):
                row['Nama Perusahaan'] = re.sub(r'(, ?\bPD\b.*)|(, ?\bPT\b.*)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD.*\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)','',text)
                row['Status Badan Usaha'] = re.search(r'(, ?\bPD\b.*)|(, ?\bPT\b)|(, ?\bCO\b.?)|(, ?\bCV\b.?)|(,? ?\bBUT\b)|(,? ?\bGMBH\b)|(,? ?\bLTD\b)|(,? ?\bPTE\b)|(, ?\bINC\b)|(,? ?\bS\.R\.L\b)|(,? ?\bS\.P\.A\b)|(,? ?\bSDN BHD\b)', text).group().replace(',','').strip(' \n\r\t')
            else:
                row['Nama Perusahaan'] = text
                if(re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bGMBH\b)|(\bBUT\b)|(\bS\.P\.A\b)',text)):
                    row['Status Badan Usaha'] = re.search(r'(\bS\.R\.L\b)|(\bSDN BHD\b)|(\bLTD\b)|(\bBUT\b)|(\bGMBH\b)|(\bS\.P\.A\b)',text).group()
                elif(re.search(r'\bLIMITED\b',text)):
                    row['Status Badan Usaha'] = 'LTD'
                elif(re.search(r'\b KAB \b|\b KOTA \b',text)):
                    row['Status Badan Usaha'] = 'PD'
                else:
                    row['Status Badan Usaha'] = 'None'
        elif(re.search(r'.*@.*\.[a-zA-Z]{2,3}\.?', text)) or (re.search(r'.*@.*', text)):
            tempEmail = text
            row['Email'] = tempEmail
        elif(re.search(r'(^www.)|(^http)|(.com$)|(.co.id$)|(.net$)', text)):
            tempWeb = text
            row['Website'] = tempWeb
        elif(re.search(r'^[0-9]{2}[a-zA-Z\s\-]+( - .*)?$', text)):
            row['Kategori'] = re.sub(r'\d+ | - .*','', text)
            row['Kode Kategori'] = re.sub(r' [a-zA-Z].*','', text)
        elif(re.search(r'(^.*(RT\.|RW\.|KEC\.|JL|KAB\.|DSN\.|JALAN |JLN|KOTA ).*$)', text)):
            row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
        else:
            row['Alamat'] = (row['Alamat']+' '+text).strip(' \n\r\t')
    data.append(row)
    tempTel1 = ''
    tempTel2 = ''
    tempTel3 = ''  
    tempEmail = ''
    tempWeb = ''

In [76]:
df = pd.DataFrame(data)

In [77]:
df

,Alamat,Email,Kategori,Kode Kategori,Nama Perusahaan,Status Badan Usaha,Telp1,Telp2,Telp3,Website
0,,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RSUD RSUD KAB. ACEH SINGKIL,None,0650-8001070 KAB. SIMEULUE 0650-8001070 KAB. S...,,,
1,,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,KLINIK DELIMA MAKMUR,PT,081260252321 KAB. SIMEULUE,,,
2,,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,KLINIK PERKEBUNAN,None,"MAKMUR, KEC. DANAU PARIS KAB. SIMEULUE 0823044...",,,
3,,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RSUD ACEH SINGKIL,None,23784 KAB. SIMEULUE 0658-21878 KAB. SIMEULUE 0...,,,
4,23784 KAB. ACEH SINGKIL,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RSUD H HASANUDIN,None,,,,
5,,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RSUD DR ZUBIR MAHMUD,None,"JL KUTACANE BELANG KEJEREN, TANAH MERAH, KAB. ...",,,
6,,rsud.idi@gmail.com KAB. ACEH TIMUR JL MEDAN BA...,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,BLUD DATU BERU TAKENGON RSUD DR ZUBIR MAHMUD K...,None,,,,
7,"JL RSUD DATU BERU, BUKIT, KEC. KEBAYAKAN KAB. ...",,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RSUD CUT NYAK DHIEN,None,,,,
8,,,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RSUS MONTELLA,None,0655-7551275 KAB. ACEH TENGAH 0655-7557274 KAB...,,,
9,,PAHLAWAN KAB. ACEH TENGAH 23617 montella_rsus@...,AKTIVITAS KESEHATAN MANUSIA DAN AKTIVITAS SOSIAL,,RUMAH SAKIT HARAPAN SEHAT,None,,,,


In [110]:
re.search(r'Ext: \b[\d]{5}\b', 'Ext: 12345') == None

False

In [112]:
'Kota D U M A I'.strip()

'Kota D U M A I'

'D U M A I'